In [70]:
import pandas as pd

df = pd.read_excel("../data/BDD Surcouche pour dataviz_v03.xlsx")#, dtype={'Groupe': str})
df = df.drop([0])

t1b4_data = df.rename(columns={'Group':'company_name'})#.astype(str)


In [71]:
# Generating company_id variable

import base64
def encode(text):
  btext = text.encode('utf-8')[:6]
  return base64.b64encode(btext).decode("utf-8") 


t1b4_data['company_id'] = [encode(x) for x in t1b4_data['company_name']]



In [72]:
# Company selection

company = 'Danone'
index = secteur = t1b4_data[t1b4_data['company_name'] == company].index[0]
secteur = t1b4_data.loc[index, 'Sector']


In [157]:
# Calculating global performance to choose pic

list_reference = ['Unrevealed', 'TotallyInsufficient', 'Sufficient', 'Partial', 'Strong', 'VeryStrong']

ref_dict = {}
for i, t in enumerate(list_reference):
    ref_dict[i+1] = t
    
global_score_reference_table = pd.DataFrame.from_dict(ref_dict, orient='index').reset_index().rename(columns={'index':'global_score',0:'global_score_label'})
global_score_reference_table['global_score'] = global_score_reference_table['global_score'].astype('str')

In [141]:
# Calculating score performance to choose color

list_reference2 = ['Non publiée', 'Vers + 4°C', 'Entre +2°C et +3°C', '2°C', 'Bien moins de 2°C', '1,5°C', 'n.a. (trop récente)']
list_hexcolors_direct = ['820000', 'C00000', 'FF8939', 'FEC800', '8CDF41', '0DB800', 'C00000']
list_scores = ['1','2','3','4','5','6','99']
    
direct_score_reference_table = pd.DataFrame(data={'direct_score': list_scores, 'direct_score_label': list_reference2, 'hexcolor':list_hexcolors_direct})

In [148]:
# Generating filtered df once company is selected

df_filtered = t1b4_data[t1b4_data['Sector']==secteur].copy()

for col in ['Global score', 'C1 direct score', 'C2 complete score']:
    df_filtered[col] = df_filtered[col].astype('int').astype('str')
    
# Filtering data & adding reference columns

df_filtered = df_filtered[['company_name','Global score','C1 direct score','C2 complete score']]
df_filtered = pd.merge(df_filtered, direct_score_reference_table, how='left', left_on='C1 direct score', right_on='direct_score').rename(columns={'direct_score_label':'c1_label', 'hexcolor': 'c1_color'}).drop(columns=['direct_score'])
df_filtered = pd.merge(df_filtered, direct_score_reference_table, how='left', left_on='C2 complete score', right_on='direct_score').rename(columns={'direct_score_label':'c2_label', 'hexcolor': 'c2_color'}).drop(columns=['direct_score'])
df_filtered = pd.merge(df_filtered, global_score_reference_table, how='left', left_on='Global score', right_on='global_score').drop(columns=['global_score'])

# Adding the global score image

df_filtered['global_score_pic'] = 'assets/frames/climate_score/Frame_'+df_filtered['global_score_label']+'.png' 

df_filtered.to_csv(path_or_buf='t1b4.csv', sep=',', header=True, index=False)

In [156]:
df_filtered

,company_name,Global score,C1 direct score,C2 complete score,c1_label,c1_color,c2_label,c2_color,global_score_label,global_score_pic
0,Associated British Foods (alim.),3,5,1,Bien moins de 2°C,8CDF41,Non publiée,820000,Sufficient,./assets/frames/climate_score/Frame_Sufficient...
1,Bel,4,3,5,Entre +2°C et +3°C,FF8939,Bien moins de 2°C,8CDF41,Partial,./assets/frames/climate_score/Frame_Partial.png
2,Danone,6,6,5,"1,5°C",0DB800,Bien moins de 2°C,8CDF41,VeryStrong,./assets/frames/climate_score/Frame_VeryStrong...
3,Ecotone,5,6,1,"1,5°C",0DB800,Non publiée,820000,Strong,./assets/frames/climate_score/Frame_Strong.png
4,Ferrero,2,2,2,Vers + 4°C,C00000,Vers + 4°C,C00000,TotallyInsufficient,./assets/frames/climate_score/Frame_TotallyIns...
5,General Mills,5,6,4,"1,5°C",0DB800,2°C,FEC800,Strong,./assets/frames/climate_score/Frame_Strong.png
6,Hero,4,3,1,Entre +2°C et +3°C,FF8939,Non publiée,820000,Partial,./assets/frames/climate_score/Frame_Partial.png
7,Kellogg,4,6,3,"1,5°C",0DB800,Entre +2°C et +3°C,FF8939,Partial,./assets/frames/climate_score/Frame_Partial.png
8,Mars,4,6,3,"1,5°C",0DB800,Entre +2°C et +3°C,FF8939,Partial,./assets/frames/climate_score/Frame_Partial.png
9,Mondelez,3,5,99,Bien moins de 2°C,8CDF41,n.a. (trop récente),C00000,Sufficient,./assets/frames/climate_score/Frame_Sufficient...
